In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from numpy import load

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

In [ ]:
X_train=[]
y_train=[]
X_val=[]
y_val=[]

os.chdir('/content/drive/MyDrive/생체인증보안/03_iris_training')
for i in os.listdir():
    img = cv2.imread(i)   
    img = cv2.resize(img,(160,160))
    X_val.append(img)
    y_val.append(i[0:3])
    X_train.append(img)
    y_train.append(i[0:3])

In [ ]:
X_test = []
y_test = []

os.chdir('/content/drive/MyDrive/생체인증보안/03_iris_test')
for i in os.listdir():
    img = cv2.imread(i)   
    img = cv2.resize(img,(160,160))
    X_test.append(img)
    y_test.append(i.rstrip('.png'))

In [ ]:
print ("Shape of an image in X_train: ", X_train[0].shape)
print ("Shape of an image in X_val: ", X_val[0].shape)
print ("Shape of an image in X_test: ", X_test[0].shape)
print("Total categories: ", len(np.unique(y_train)))
print("Total categories: ", len(np.unique(y_val)))
print("Total categories: ", len(np.unique(y_test)))

In [ ]:
le = preprocessing.LabelEncoder()

y_train = le.fit_transform(y_train)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=64)
y_train = np.array(y_train)

y_val = le.fit_transform(y_val)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=64)
y_val = np.array(y_val)

X_train = np.array(X_train)
X_test = np.array(X_test)
X_val=np.array(X_val)

In [ ]:
from numpy import asarray
print("X_train Shape: ", X_train.shape)
print("y_train Shape: ", y_train.shape)

print("X_val Shape: ", X_val.shape)
print("y_val Shape: ", y_val.shape)

print("y_test Shape: ", X_test.shape)

In [ ]:
from imgaug import augmenters as iaa

seq = iaa.Sequential([
    # blur images with a sigma of 0 to 0.5
    iaa.GaussianBlur(sigma=(0, 0.5)),
    iaa.Affine(
        # scale images to 90-110% of their size, individually per axis
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        # translate by -10 to +10 percent (per axis)
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        # if mode is constant, use a cval between 0 and 255
        cval=255,
    )
], random_order=True)

x=X_train
y=y_train
aug=40

for i in range(aug):
	x_aug=seq.augment_images(x)
	X_train=np.append(X_train, x_aug, axis=0)
	y_train=np.append(y_train, y, axis=0)

In [ ]:
from numpy import asarray
print("X_train Shape: ", X_train.shape)
print("y_train Shape: ", y_train.shape)

print("X_val Shape: ", X_val.shape)
print("y_val Shape: ", y_val.shape)

print("y_test Shape: ", X_test.shape)

# print("X_train_aug Shape: ", X_train_aug.shape)
# print("y_train_aug Shape: ", y_train_aug.shape)

In [ ]:
from keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers import LeakyReLU


# 기존 코드 수정
def get_model():
    m1=Sequential()
    m1.add(BatchNormalization(input_shape = (160,160,3)))
    m1.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (160, 160, 3))) 
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Dropout(0.1))

    m1.add(Convolution2D(filters=64,kernel_size=3,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Dropout(0.1))

    m1.add(Convolution2D(filters=128,kernel_size=3,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Dropout(0.1))

    m1.add(Convolution2D(filters=256,kernel_size=3,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))

    m1.add(Flatten()) 
    m1.add(Dense(units=160,activation = 'relu'))
    m1.add(BatchNormalization())
    m1.add(Dense(units = 64, activation = 'softmax'))
    m1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', precision, recall, f1score])  
    return m1

In [ ]:
# from sklearn.model_selection import KFold 
# from sklearn.model_selection import cross_val_score

##cross validation
# # kfold = KFold(n_splits=5, random_state = 2021, shuffle=True)
# # acc_list = [] 
# # for train_index, val_index in kfold.split(X_train):
# #     x_train_fold, x_val_fold = X_train[train_index], X_train[val_index]
# #     y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
# #     model = get_model()
# #     model.fit(x_train_fold, y_train_fold, epochs = 20, validation_data = (x_val_fold, y_val_fold))

# #     _, acc = model.evaluate(X_val,y_val)
# #     acc_list.append(round(acc,4))

model=get_model()
model.summary()
model.fit(X_train, y_train, epochs=100, validation_data=(X_val,y_val))

In [ ]:
# # 새로운 코드
# # Resnet50
# from keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2, preprocess_input, decode_predictions
# from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
# from keras.applications.resnet50 import ResNet50
# from keras.layers import Dense, Input, Activation
# from keras.models import Model
# from keras.callbacks import EarlyStopping
# from keras import optimizers
# from keras.preprocessing.image import ImageDataGenerator
# from keras.layers.normalization import BatchNormalization
# from keras.layers import MaxPooling2D

# input = Input(shape=(160, 160, 3))

# def get_model():
#     resnet=ResNet50V2(include_top=False, weights='imagenet', input_shape=input)
#     m2=resnet.output
#     m2=MaxPooling2D(pool_size=(2,2))(m2)
#     m2=Flatten()(m2)
#     m2=Dropout(0.5)(m2)
#     m2=Dense(512,activation='relu', input_dim=input)(m2)
#     m2=BatchNormalization()(m2)
#     m2=Dense(256,activation='relu')(m2)
#     m2=BatchNormalization()(m2)
#     m2=Dense(64, activation='softmax')(m2)
#     m2=Model(inputs=resnet.input, outputs=m2)
#     m2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', precision, recall, f1score])  
#     return m2


In [ ]:
# # 새로운 코드 - RESNET50

# from keras.applications.resnet50 import ResNet50
# from keras.layers import Dense, Input, Activation
# from keras.models import Model
# from keras.callbacks import EarlyStopping
# from keras import optimizers
# from keras.preprocessing.image import ImageDataGenerator
# from keras.layers.normalization import BatchNormalization
# from keras.layers import GlobalAveragePooling2D

# input = Input(shape=(160, 160, 3))
# model = ResNet50V2(input_tensor=input, include_top=False, weights='imagenet', pooling='max')

# for layer in model.layers[:44]:
#     layer.trainable=False

# x = model.output
# x = BatchNormalization()(x)
# # x = GlobalAveragePooling2D()(x)
# x = Dropout(0.2)(x)
# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.2)(x)
# x = Dense(64, activation='softmax', name='softmax')(x)
# model = Model(model.input, x)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', precision, recall, f1score])  

In [ ]:
# # model=get_model()
# # model.summary()
# model.fit(X_train, y_train, epochs=100, validation_data=(X_val,y_val))

In [ ]:
predicted = model.predict(X_test, batch_size=128)
predicted = np.argmax(np.round(predicted),axis=1)
predicted=predicted.tolist()
for i in range(256):
    predicted[i]+=1
print("Prediction :",predicted)

print(y_test)

In [ ]:
y_res=[]
y_res = map(int, y_test)

In [ ]:
result = { name:value for name, value in zip(y_res, predicted) }
result = sorted(result.items())

print(result)

In [ ]:
import csv
with open('1871080_이혜승_홍채_2차_답안.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(result)